# 第4回 宿題

## 課題：Tranformerを用いた機械翻訳

- 訓練データ（train_X, train_Y）でモデル訓練させた後、テストデータ（test_X）に対する翻訳文を生成し、結果をcsvファイルに出力してください。
- csvファイルは`sample_submission.csv`と同様に、一行に一文を、単語をスペースで分割して書き込んでください。
- ファイル名はsubmission.csvとしてください。
- 予測結果のtest_Yに対する精度（BLEU）で評価します。

In [ ]:
! head sample_submission.csv

In [ ]:
# サンプルコード

import numpy as np
import csv
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from nltk import bleu_score
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
try:
    from utils import Vocab
except ModuleNotFoundError:  # iLect環境
    import os
    os.chdir('/root/userspace/chap4/')
    from utils import Vocab

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(1)
random_state = 42

PAD = 0
UNK = 1
BOS = 2
EOS = 3
PAD_TOKEN = '<PAD>'
UNK_TOKEN = '<UNK>'
BOS_TOKEN = '<S>'
EOS_TOKEN = '</S>'


def load_data(file_path):
    data = []
    for line in open(file_path, encoding='utf-8'):
        words = line.strip().split()  # スペースで単語を分割
        data.append(words)
    return data


def load_dataset():
    # Load dataset
    train_X = load_data('./data/train.en')
    train_Y = load_data('./data/train.ja')
    test_X = load_data('./data/test.en')
    
    return train_X, train_Y, test_X


class ScaledDotProductAttention(nn.Module):
    # WRITE ME!


class MultiHeadAttention(nn.Module):
    # WRITE ME!


class PositionwiseFeedForward(nn.Module):
    # WRITE ME!


def position_encoding_init(n_position, d_pos_vec):
    # WRITE ME!


def get_attn_padding_mask(seq_q, seq_k):
    # WRITE ME!


def get_attn_subsequent_mask(seq):
    # WRITE ME!


class EncoderLayer(nn.Module):
    # WRITE ME!


class Encoder(nn.Module):
    # WRITE ME!


class DecoderLayer(nn.Module):
    # WRITE ME!


class Decoder(nn.Module):
    # WRITE ME!


class Transformer(nn.Module):
    # WRITE ME!


def sentence_to_ids(vocab, sentence):
    """
    単語のリストをインデックスのリストに変換する
    :param vocab: Vocabのインスタンス
    :param sentence: list of str
    :return indices: list of int
    """
    ids = [vocab.word2id.get(word, UNK) for word in sentence]
    ids = [BOS] + ids + [EOS]  # </S>トークンを末尾に加える
    return ids


class DataLoader(object):
    # WRITE ME!


def compute_loss(batch_X, batch_Y, model, criterion, optimizer=None, is_train=True):
    model.train(is_train)

    pred_Y = model(batch_X, batch_Y)
    gold = batch_Y[0][:, 1:].contiguous()
    loss = criterion(pred_Y.view(-1, pred_Y.size(2)), gold.view(-1))

    if is_train:  # 訓練時はパラメータを更新
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    gold = gold.data.cpu().numpy().tolist()
    pred = pred_Y.max(dim=-1)[1].data.cpu().numpy().tolist()

    return loss.item(), gold, pred


def calc_bleu(refs, hyps):
    refs = [[ref[:ref.index(EOS)]] for ref in refs]
    hyps = [hyp[:hyp.index(EOS)] if EOS in hyp else hyp for hyp in hyps]
    return 100 * bleu_score.corpus_bleu(refs, hyps)


def test(model, src, max_length=20):
    # 学習済みモデルで系列を生成する
    model.eval()

    src_seq, src_pos = src
    batch_size = src_seq.size(0)
    enc_output, enc_slf_attns = model.encoder(src_seq, src_pos)

    tgt_seq = torch.LongTensor(batch_size, 1).fill_(BOS).to(device)
    tgt_pos = torch.arange(1).unsqueeze(0).repeat(batch_size, 1)
    tgt_pos = tgt_pos.type(torch.LongTensor).to(device)

    # 時刻ごとに処理
    for t in range(1, max_length+1):
        dec_output, dec_slf_attns, dec_enc_attns = model.decoder(
            tgt_seq, tgt_pos, src_seq, enc_output)
        dec_output = model.tgt_word_proj(dec_output)
        out = dec_output[:, -1, :].max(dim=-1)[1].unsqueeze(1)
        # 自身の出力を次の時刻の入力にする
        tgt_seq = torch.cat([tgt_seq, out], dim=-1)
        tgt_pos = torch.arange(t+1).unsqueeze(0).repeat(batch_size, 1)
        tgt_pos = tgt_pos.type(torch.LongTensor).to(device)

    return tgt_seq[:, 1:], enc_slf_attns, dec_slf_attns, dec_enc_attns


train_X, train_Y, test_X = load_dataset()

train_X, valid_X, train_Y, valid_Y = train_test_split(
    train_X, train_Y, test_size=0.1, random_state=42)

word2id = {
    PAD_TOKEN: PAD,
    BOS_TOKEN: BOS,
    EOS_TOKEN: EOS,
    UNK_TOKEN: UNK,
    }

vocab_X = Vocab(word2id=word2id)
vocab_Y = Vocab(word2id=word2id)
vocab_X.build_vocab(train_X, min_count=MIN_COUNT)
vocab_Y.build_vocab(train_Y, min_count=MIN_COUNT)

vocab_size_X = len(vocab_X.id2word)
vocab_size_Y = len(vocab_Y.id2word)

train_X = [sentence_to_ids(vocab_X, sentence) for sentence in train_X]
train_Y = [sentence_to_ids(vocab_Y, sentence) for sentence in train_Y]
valid_X = [sentence_to_ids(vocab_X, sentence) for sentence in valid_X]
valid_Y = [sentence_to_ids(vocab_Y, sentence) for sentence in valid_Y]

train_dataloader = DataLoader(train_X, train_Y, BATCH_SIZE)
valid_dataloader = DataLoader(valid_X, valid_Y, BATCH_SIZE, 
                              is_training=False, shuffle=False)

BATCH_SIZE = 64
MIN_COUNT = 2
MAX_LENGTH = 20
NUM_EPOCHS = 1
max_len = MAX_LENGTH + 2

model_args = {
    'n_src_vocab': vocab_size_X,
    'n_tgt_vocab': vocab_size_Y,
    'max_len': max_len,
    'proj_share_weight': True,
    'd_k': 32,
    'd_v': 32,
    'd_model': 128,
    'd_word_vec': 128,
    'd_inner_hid': 256,
    'n_layers': 3,
    'n_head': 6,
    'dropout': 0.1,
}
model = Transformer(**model_args).to(device)
optimizer = optim.Adam(model.get_trainable_parameters())
criterion = nn.CrossEntropyLoss(ignore_index=PAD, size_average=False).to(device)    

# 訓練
best_valid_bleu = 0.

for epoch in range(1, NUM_EPOCHS+1):
    train_loss = 0.
    train_refs = []
    train_hyps = []
    valid_loss = 0.
    valid_refs = []
    valid_hyps = []
    # train
    for batch in train_dataloader:
        batch_X, batch_Y = batch
        loss, gold, pred = compute_loss(
            batch_X, batch_Y, model, criterion, optimizer, is_train=True
            )
        train_loss += loss
        train_refs += gold
        train_hyps += pred
    # valid
    for batch in valid_dataloader:
        batch_X, batch_Y = batch
        loss, gold, pred = compute_loss(
            batch_X, batch_Y, model, criterion, is_train=False
            )
        valid_loss += loss
        valid_refs += gold
        valid_hyps += pred
    # 損失をサンプル数で割って正規化
    train_loss /= len(train_dataloader.data) 
    valid_loss /= len(valid_dataloader.data) 
    # BLEUを計算
    train_bleu = calc_bleu(train_refs, train_hyps)
    valid_bleu = calc_bleu(valid_refs, valid_hyps)

    # validationデータでBLEUが改善した場合にはモデルを保存
    if valid_bleu > best_valid_bleu:
        ckpt = model.state_dict()
        best_valid_bleu = valid_bleu

    print('Epoch {}: train_loss: {:5.2f}  train_bleu: {:2.2f}  valid_loss: {:5.2f}  valid_bleu: {:2.2f}'.format(
            epoch, train_loss, train_bleu, valid_loss, valid_bleu))
    print('-'*80)

# 学習済みモデルで生成
model.load_state_dict(ckpt)  # 最良のモデルを読み込み
# （一時的に）Yを使わずにDataLoaderを利用する
test_X = [sentence_to_ids(vocab_X, sentence) for sentence in test_X]
test_dataloader = DataLoader(test_X, test_X, BATCH_SIZE, 
                              is_training=False, shuffle=False)    

pred_Y = []
for batch in test_dataloader:
    batch_X, _ = batch
    preds, *_ = test(model, batch_X)
    preds = preds.data.cpu().numpy().tolist()
    preds = [pred[:pred.index(EOS)] if EOS in pred else pred for pred in preds]
    pred_y = [[vocab_Y.id2word[_id] for _id in pred] 
                     for pred in preds]
    pred_Y += pred_y


with open('submission.csv', 'w') as f:
    writer = csv.writer(f, delimiter=' ', lineterminator='\n')
    writer.writerows(pred_Y)

In [ ]:
save_homework()